In [ ]:
#Import requisite modules
import sys
import pandas as pd
import datetime
from twitterscraper import query_tweets #if you haven't installed this module, run 'pip install twitterscraper' in your notebook

query_list = [ #This is our sample list, add or subtract as you see fit!
    'COVID',
    'COVID-19',
    'Corona',
    'Coronavirus',
    'Rona',
    'Quarantine',
    '#COVID',
    '#COVID-19',
    '#quarantine',
    '#Quarantine',
    '#covid19'
]
#Credit to Danielle Medellin, DSI11-NYC for the below implementation of custom parameter dictionary support
custom_params = {'Houston':{'city' : 'Houston',
                         'lat'  : 29.760427,
                         'long' : -95.369804,
                         'radius': '15mi',
                         'queries' : ['list','random','words']},
              'Detroit': {'city' : 'Detroit',
                          'lat'  : 42.331429,
                          'long' : -83.045753,
                          'radius' : '10mi',
                         'queries': ['more','weird','words']}
             }

#Get tweets without geolocation
def get_tweets_no_geoloc(query): 
    tweets = {}
    count = 0 #Sets the index generator
    for tweet in query_tweets(query,10,begindate=datetime.date(2019,12,1)): #Runs 100 pings, 
        chirp = {}
        chirp['tweet_id'] = tweet.tweet_id
        chirp['username'] = tweet.username
        chirp['text'] = tweet.text
        chirp['tweet_date'] = tweet.timestamp
        chirp['search_term'] = query
        chirp['long'] = 'n/a' #fills in n/a value for all geolocation columns to avoid feature issues later.
        chirp['lat'] = 'n/a'
        chirp['radius'] = 'n/a'
        tweets.update({count : chirp})
        count += 1
    return tweets

#Get tweets with geolocation
def get_tweets_geoloc(query, lat, long, radius): #Geolocation parameters defined by user in master function
    tweets = {}
    count = 0
    for tweet in query_tweets(f"{query}, geocode:{lat},{long},{radius}",10,begindate=datetime.date(2019,12,1)):
        chirp = {} #Generates tweet dictionary by calling on generated 'tweet' object attributes
        chirp['tweet_id'] = tweet.tweet_id
        chirp['username'] = tweet.username
        chirp['text'] = tweet.text
        chirp['tweet_date'] = tweet.timestamp
        chirp['search_term'] = query
        chirp['lat'] = lat
        chirp['long'] = long
        chirp['radius'] = radius
        tweets.update({count : chirp})
        count += 1 #increments index up by 1 for later dataframe implementation
    return tweets

#Generate dataframe from "tweets" dictionary generated after each query
def make_dataframe(dictionary):
    df = pd.DataFrame.from_dict(dictionary, orient='index') 
    return df
#--------------------------------------------------------------------
#Master function
def get_query_dataframe(list_of_queries):
    geo_switch = input("Are you using geolocation?")
    dataset = pd.DataFrame() #instantiate an empty dataframe
    
    #These parameters are under development
#     year = int(input("Query start year:"))
#     month = int(input("Query start month:"))
#     day = int(input("Query start day:"))
   
    if str.lower(geo_switch) == 'yes': #Tests if lat, long, and radius inputs are needed
        lat = float(input("Input Latitude:")) #Converts string input latitude to float value
        long = float(input("Input Longitude:"))
        radius = input("Input radius and unit:")
    else:
        pass
    
    if str.lower(geo_switch) == 'yes': #Sets function to run with geoloc terms in query
        for query in list_of_queries:
            tweets = get_tweets_geoloc(query,lat,long,radius)
            df = make_dataframe(tweets)
            dataset = pd.concat([dataset,df],ignore_index = True)
    else:
        for query in list_of_queries:
            tweets = get_tweets_no_geoloc(query)
            df = make_dataframe(tweets)
            dataset = pd.concat([dataset,df],ignore_index = True)
    return dataset